In [1]:
# Display output of plots directly in Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import intake
import numpy as np
import pandas as pd
import xarray as xr

from dask.distributed import Client
from dask_cuda import LocalCUDACluster


In [ ]:
cluster = LocalCUDACluster()
client = Client(cluster)
client

In [ ]:
# Open collection description file
catalog_url = 'https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le.json'
col = intake.open_esm_datastore(catalog_url)
col

In [ ]:
col.df.head(1000)

In [ ]:
# Show expanded version of collection structure with details
import pprint
uniques = col.unique(columns=["component", "frequency", "experiment", "variable"])
pprint.pprint(uniques, compact=True, indent=4)

In [ ]:
col_subset = col.search(frequency=["daily","monthly"], component="lnd",
                        #variable=["SST"],
                        experiment=["RCP85"])

col_subset

In [ ]:
col_subset.df

In [ ]:
# Load catalog entries for subset into a dictionary of xarray datasets
#!pip install s3fs
dsets = col_subset.to_dataset_dict(zarr_kwargs={"consolidated": True, 'use_cftime': True, 'decode_times': True}, storage_options={"anon": True})
print(f"\nDataset dictionary keys:\n {dsets.keys()}")

In [ ]:
# Define Xarray datasets corresponding to the three experiments
ds_RCP85 = dsets['lnd.RCP85.daily']
ds_RCP85

In [ ]:
dsets

In [ ]:
# Use Dask.Distributed utility function to display size of each dataset
from distributed.utils import format_bytes
print(f"RCP8.5: {format_bytes(ds_RCP85.nbytes)}")

In [ ]:
len(ds_RCP85["FSNO"].time.data)

In [ ]:
# Extract the Reference Height Temperature data variable
t_rcp_FSNO = ds_RCP85["FSNO"]
t_rcp_FSNO

In [ ]:
t_rcp_FSNO.time.data

In [ ]:
# The global surface temperature anomaly was computed relative to the 1961-90 base period
# in the Kay et al. paper, so extract that time slice
t_ref = t_rcp_FSNO.sel(time=slice("2006"))
t_ref

In [ ]:
!python --version

In [ ]:
catalog_url = "https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le.json"
col = intake.open_esm_datastore(catalog_url)
col

In [ ]:
col_subset = col.search(
    frequency=["daily"],
    component="atm",
    variable=["TREFHT","QBOT"],
    experiment=["RCP85"],
)

col_subset

In [ ]:
col_subset.df

In [ ]:
dsets = col_subset.to_dataset_dict(zarr_kwargs={"consolidated": True, 'use_cftime': True, 'decode_times': True}, storage_options={"anon": True})

In [ ]:
ds = dsets['atm.RCP85.daily']
ds

In [ ]:
lat = ds.lat.data
nlat = len(lat)
lon = ds.lon.data
nlon = len(lon)

In [ ]:
import pickle
elevation = pickle.load(open("elevation.pkl", "rb"))
lat = lat[30:160]
nlat = len(lat)
elevation = elevation[:][30:160]

In [2]:
import geocat.comp as gc
print(gc.relhum.__doc__)
print(gc.dewtemp.__doc__)

This function calculates the relative humidity given temperature, mixing
    ratio, and pressure.

    "Improved Magnus' Form Approx. of Saturation Vapor pressure"
    Oleg A. Alduchov and Robert E. Eskridge
    https://www.osti.gov/scitech/servlets/purl/548871/
    https://doi.org/10.2172/548871

    Parameters
    ----------
    temperature : :class:`numpy.ndarray`, :class:`xarray.DataArray`, :obj:`list`, or :obj:`float`
        Temperature in Kelvin

    mixing_ratio : :class:`numpy.ndarray`, :class:`xarray.DataArray`, :obj:`list`, or :obj:`float`
        Mixing ratio in kg/kg. Must have the same dimensions as temperature

    pressure : :class:`numpy.ndarray`, :class:`xarray.DataArray`, :obj:`list`, or :obj:`float`
        Pressure in Pa. Must have the same dimensions as temperature

    Returns
    -------
    relative_humidity : :class:`numpy.ndarray` or :class:`xarray.DataArray`
        Relative humidity. Will have the same dimensions as temperature


    See Also
    --------
 

In [8]:
from ncar_jobqueue import NCARCluster
import dask.distributed as dd
cluster = NCARCluster()
cluster.scale(jobs=1)
client = dd.Client(cluster)

In [9]:
client

<Client: 'tcp://10.12.205.44:34394' processes=2 threads=4, memory=46.56 GiB>

In [11]:
client.cancel(cluster)